In [ ]:
import requests
import json
import requests_cache
import time
from IPython.core.display import clear_output
import pandas as pd
import numpy as np
from tqdm import tqdm

requests_cache.install_cache()

In [ ]:
API_KEY = 'a2ba28906c69f6372a0e34f7b3c5f6d3'
USER_AGENT = 'infoquest'

In [ ]:
def lastfm_get(payload):
    # define headers and URL
    headers = {
        'user-agent': USER_AGENT
    }
    url = 'http://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

def to_json(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    return text

## Chart

### 1.1 Get Top Artists

In [ ]:
responses = []
page = 1
total_pages = 99999

while page <= total_pages:
    payload = {
        'method': 'chart.gettopartists',
        'limit': 500,
        'page': page
    }    
    print("Requesting page {}/{}".format(page, total_pages))
    clear_output(wait = True)
    
    response = lastfm_get(payload)
    
    if response.status_code != 200:
        print(response.text)
        break

    page = int(response.json()['artists']['@attr']['page'])
    total_pages = int(response.json()['artists']['@attr']['totalPages'])
    responses.append(response)

    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)        
    page += 1

In [ ]:
def lookup_tags(artist):
    response = lastfm_get({
        'method': 'artist.getTopTags',
        'artist':  artist
    })

    if response.status_code != 200:
        return None

    # extract the top three tags and turn them into a string
    tags = [t['name'] for t in response.json()['toptags']['tag'][:3]]
    tags_str = ', '.join(tags)

    # rate limiting
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return tags_str

In [ ]:
frames = [pd.DataFrame(r.json()['artists']['artist']) for r in responses]
artists = pd.concat(frames)
tqdm.pandas()
artists['tags'] = artists['name'].progress_apply(lookup_tags)

In [ ]:
artists

In [ ]:
artists.to_csv('./app/data/artists.csv')

### 1.2 Get Top Tags

In [ ]:
r = lastfm_get({
    'method': 'artist.getTopTags',
    'artist':  'Lana Del Rey'
})

In [ ]:
r = lastfm_get({
    'method': 'user.getWeeklyAlbumChart',
    'user': 'Efoja_87',
    'from': '1212321600',
    'to': '1212926400'
})

r.json()

In [ ]:
r = lastfm_get({
    'method': 'user.getWeeklyAlbumChart',
    'user': 'bisa',
    'from': '1212321600',
    'to': '1212926400'
})

r.json()

In [ ]:
r = lastfm_get({
    'method': 'user.getWeeklyChartList',
    'user': 'bisa',
})

r.json()